<a href="https://colab.research.google.com/github/SanthoshToorpu/AI-plays-Minecraft/blob/main/Finetune_LLMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

qwen_models = [
    "unsloth/Qwen2.5-Coder-32B-Instruct",      # Qwen 2.5 Coder 2x faster
    "unsloth/Qwen2.5-Coder-7B",
    "unsloth/Qwen2.5-14B-Instruct",            # 14B fits in a 16GB card
    "unsloth/Qwen2.5-7B",
    "unsloth/Qwen2.5-72B-Instruct",            # 72B fits in a 48GB card
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.8: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import json

with open('/content/gpt_conversation_dataset.json', 'r') as f:
    data = json.load(f)

from datasets import Dataset

flattened_data = [item for sublist in data for item in sublist]

# Group the flattened data into the original conversation format
conversations = []
current_convo = []
for entry in flattened_data:
    current_convo.append(entry)

    if entry['from'] == 'gpt':
        conversations.append(current_convo)
        current_convo = []

# Add the last conversation if it doesn't end with a 'gpt' turn
if current_convo:
    conversations.append(current_convo)


# Create dummy data for other columns
num_rows = len(conversations)
sources = ["your_source"] * num_rows # Replace with actual sources if available
scores = [0] * num_rows # Replace with actual scores if available

# Create the dataset
dataset = Dataset.from_dict({
    "conversations": conversations,
})

dataset

Dataset({
    features: ['conversations'],
    num_rows: 1580
})

In [ ]:
from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/1580 [00:00<?, ? examples/s]

In [ ]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/1580 [00:00<?, ? examples/s]

In [ ]:
dataset[5]

{'conversations': [{'content': 'Write a code toMine wood log at daytime',
   'role': 'user'},
  {'content': 'async function mineWoodLogAtDaytime(bot) {\n const woodLogNames = ["oak_log", "birch_log", "spruce_log", "jungle_log", "acacia_log", "dark_oak_log", "mangrove_log"];\n if (bot.time.timeOfDay > 1000 && bot.time.timeOfDay < 12000) {\n const woodLogBlock = await exploreUntil(bot, new Vec3(1,0,1),60, () => {\n return bot.findBlock({\n matching: block => woodLogNames.includes(block.name),\n maxDistance:32\n });\n });\n if (!woodLogBlock) {\n bot.chat("Could not find a wood log.");\n return;\n }\n await mineBlock(bot, woodLogBlock.name,1);\n bot.chat("Wood log mined.");\n } else {\n bot.chat("It is not daytime.");\n }\n}',
   'role': 'assistant'}],
 'text': '<|begin_of_text|>Below are some instructions that describe some tasks. Write responses that appropriately complete each request.\n\n### Instruction:\nWrite a code toMine wood log at daytime\n\n### Response:\nasync function mineWoo

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        max_steps = 800,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 3e-4,
        logging_steps = 25,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        save_steps = 200,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1580 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|>Below are some instructions that describe some tasks. Write responses that appropriately complete each request.\n\n### Instruction:\nWrite a code toMine wood log at daytime\n\n### Response:\nasync function mineWoodLogAtDaytime(bot) {\n const woodLogNames = ["oak_log", "birch_log", "spruce_log", "jungle_log", "acacia_log", "dark_oak_log", "mangrove_log"];\n if (bot.time.timeOfDay > 1000 && bot.time.timeOfDay < 12000) {\n const woodLogBlock = await exploreUntil(bot, new Vec3(1,0,1),60, () => {\n return bot.findBlock({\n matching: block => woodLogNames.includes(block.name),\n maxDistance:32\n });\n });\n if (!woodLogBlock) {\n bot.chat("Could not find a wood log.");\n return;\n }\n await mineBlock(bot, woodLogBlock.name,1);\n bot.chat("Wood log mined.");\n } else {\n bot.chat("It is not daytime.");\n }\n}<|eot_id|>'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.455 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,580 | Num Epochs = 5 | Total steps = 800
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
25,0.771100
50,0.452500
75,0.279000
100,0.241300
125,0.208500
150,0.170700
175,0.165500
200,0.159000
225,0.137800
250,0.133700


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2695.4916 seconds used for training.
44.92 minutes used for training.
Peak reserved memory = 3.475 GB.
Peak reserved memory for training = 0.02 GB.
Peak reserved memory % of max memory = 23.574 %.
Peak reserved memory for training % of max memory = 0.136 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Write a code toMine a wood log"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 1024, pad_token_id = tokenizer.eos_token_id)

async function mineWoodLog(bot) {
 const logNames = ["oak_log", "birch_log", "spruce_log", "jungle_log", "acacia_log", "dark_oak_log", "mangrove_log"];
 const logBlock = await exploreUntil(bot, new Vec3(1,0,1),60, () => {
 for (const logName of logNames) {
 const log = bot.findBlock({
 matching: mcData.blocksByName[logName].id,
 maxDistance:32
 });
 if (log) return log;
 }
 return null;
 });
 if (!logBlock) {
 bot.chat("Could not find a wood log.");
 return;
 }
 await mineBlock(bot, logBlock.name,1);
 bot.chat("Wood log mined.");
}<|eot_id|>


In [ ]:
from huggingface_hub import login

# Step 1: Login to Hugging Face Hub

print("✅ Successfully logged in to Hugging Face Hub!")

# Step 2: Push model and tokenizer to hub
try:
    print("📤 Pushing model to hub...")
    model.push_to_hub("SanthoshToorpu/Llama-3.2-Minecraft")
    print("✅ Model pushed successfully!")

    print("📤 Pushing tokenizer to hub...")
    tokenizer.push_to_hub("SanthoshToorpu/Llama-3.2-Minecraft")
    print("✅ Tokenizer pushed successfully!")

    print("\n🎉 Model and tokenizer successfully uploaded to:")
    print("https://huggingface.co/SanthoshToorpu/Llama-3.2-Minecraft")

except Exception as e:
    print(f"❌ Error occurred: {str(e)}")
    print("Please check your internet connection and try again.")

✅ Successfully logged in to Hugging Face Hub!
📤 Pushing model to hub...


README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/SanthoshToorpu/Llama-3.2-Minecraft
✅ Model pushed successfully!
📤 Pushing tokenizer to hub...


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

✅ Tokenizer pushed successfully!

🎉 Model and tokenizer successfully uploaded to:
https://huggingface.co/SanthoshToorpu/Llama-3.2-Minecraft
